In [7]:
using RigidBodySim
using RigidBodyDynamics
using MechanismGeometries

In [2]:
urdf = "shtuff\\doublependulum.urdf"
mechanism = parse_urdf(Float64, urdf)
# Vertexuuuus

Spanning tree:
Vertex: world (root)
  Vertex: base_link, Edge: base_link_to_world
    Vertex: upper_link, Edge: shoulder
      Vertex: lower_link, Edge: elbow
No non-tree joints.

In [32]:
state = MechanismState(mechanism)
shoulder, elbow = joints(mechanism)
configuration(state, shoulder) .= 0.3
configuration(state, elbow) .= 0.4
velocity(state, shoulder) .= 1.
velocity(state, elbow) .= 2.;

In [33]:
open_loop_dynamics = Dynamics(mechanism);
final_time = 1000.
problem = ODEProblem(open_loop_dynamics, state, (0., final_time))

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 1000.0)
u0: [0.4, 0.0, 2.0, 0.0]

In [9]:
visuals = URDFVisuals(urdf);

In [10]:
gui = GUI(mechanism, visuals)
open(gui); # This will open a cheeky gui (might take a while to load)

In [ ]:
# or:

# using Blink: Window
# open(gui.controls, Window())
# open(gui.visualizer, Window());

In [34]:
vis = gui.visualizer # a MeshCatMechanisms.MechanismVisualizer
set_configuration!(vis, configuration(state));

In [35]:
gui_callback = CallbackSet(gui);

In [36]:
sol = solve(problem, Tsit5(), abs_tol = 1e-10, dt = 0.05,
    callback = gui_callback);
# Watch as the pendulum tries to lick its own markov blanket

In [21]:
function control!(τ, t, state)
    view(τ, velocity_range(state, shoulder))  .= 5 * sin(t)
    view(τ, velocity_range(state, elbow)) .= -configuration(state, shoulder)
end

control! (generic function with 1 method)